In [ ]:
import os
import pandas as pd
import pdr

def bytes_to_int(v):
    if isinstance(v, int):
        return v
    if isinstance(v, float):
        return int(v)
    if isinstance(v, bytes):
        return int.from_bytes(v, "big", signed=False)
    if isinstance(v, str):
        return int.from_bytes(v.encode("latin1"), "big", signed=False)
    return 0

odf_folder = "input_data"
odf_files = [f for f in os.listdir(odf_folder) if f.endswith(".odf")]

all_summary = []
all_orbit = []
all_ramp = []

ramp_ids = [15, 45, 54, 65]

for odf_file in odf_files:
    odf_path = os.path.join(odf_folder, odf_file)
    lbl_file = odf_file.replace(".odf", ".lbl")
    lbl_path = os.path.join(odf_folder, lbl_file)

    odf_data = pdr.read(odf_path, lbl_path)

    # Data Summary Group DATA (основная сводка наблюдений)
    # Когда, какая станция, сколько сэмплов
    if "ODF7B_TABLE" in odf_data: 
        table = odf_data["ODF7B_TABLE"]
        for row in table:
            if len(row) < 9:
                continue
            obs = {
                "t_start_sec": bytes_to_int(row[0]),
                "t_start_frac": bytes_to_int(row[1]),
                "station_id": bytes_to_int(row[2]),
                "doppler_channel": bytes_to_int(row[3]),
                "band_id": bytes_to_int(row[4]),
                "data_type": bytes_to_int(row[5]),
                "n_samples": bytes_to_int(row[6]),
                "t_end_sec": bytes_to_int(row[7]),
                "t_end_frac": bytes_to_int(row[8])
            }
            all_summary.append(obs)

    # Orbit Data Group DATA (основные телекоммуникационные наблюдения)
    # Доплер, время, фракции
    if "ODF3C_TABLE" in odf_data: 
        table = odf_data["ODF3C_TABLE"]
        for row in table:
            if len(row) < 4:
                continue

            time_sec = bytes_to_int(row[0])
            frac_bits = bytes_to_int(row[1])
            observable_int = bytes_to_int(row[2])
            observable_frac = bytes_to_int(row[3])

            orbit = {
                "time_sec": time_sec,
                "time_frac_ms": frac_bits & 0x3FF,
                "downlink_delay_ns": (frac_bits >> 10) & 0x3FFFFF,
                "observable_int": observable_int,
                "observable_frac": observable_frac
            }
            all_orbit.append(orbit)

    # Ramp Group x DATA (частотные настройки трансиверов)
    # Настройка частот на станциях DSN №x
    for x in ramp_ids:
        table_name = f"ODF4B{x}_TABLE"
        if table_name not in odf_data:
            continue

        table = odf_data[table_name]
        for row in table:
            if len(row) < 9:
                continue

            ramp = {
                "ramp_group_id": x,
                "ramp_start_sec": bytes_to_int(row[0]),
                "ramp_start_frac": bytes_to_int(row[1]),
                "ramp_rate_int": bytes_to_int(row[2]),
                "ramp_rate_frac": bytes_to_int(row[3]),
                "station_id": bytes_to_int(row[4]) & 0x3FF,
                "ramp_start_freq_int": bytes_to_int(row[5]),
                "ramp_start_freq_frac": bytes_to_int(row[6]),
                "ramp_end_sec": bytes_to_int(row[7]),
                "ramp_end_frac": bytes_to_int(row[8])
            }
            all_ramp.append(ramp)


pd.DataFrame(all_summary).to_csv("mgs_moi_odf7b.csv", index=False, sep=';')
pd.DataFrame(all_orbit).to_csv("mgs_moi_odf3c.csv", index=False, sep=';')
pd.DataFrame(all_ramp).to_csv("mgs_moi_odf4b_all.csv", index=False, sep=';')

print("Summary записей:", len(all_summary))
print("Orbit записей:", len(all_orbit))
print("Ramp записей:", len(all_ramp))


Summary записей: 64
Orbit записей: 48
Ramp записей: 189
